Completion Author: Hariharan Sezhiyan - April 30, 2018
Note: For the same of clarity, I kept all print statements. This will allow the reader to visualize datatables as he/she runs the cells. For review purposes, feel free to comment them out.

# Project: Hypothesis Testing for Microtransactions
Brian is a Product Manager at FarmBurg, a company that makes a farming simulation social network game.  In the FarmBurg game, you can plow, plant, and harvest different crops.

Today, you will be acting as Brian's data analyst for an A/B Test that he has been conducting.

## Part 1: Testing for Significant Difference

Start by importing the following modules that you'll need for this project:
- `pandas` as `pd`

In [1]:
import pandas as pd

Brian tells you that he ran an A/B test with three different groups: A, B, and C.  You're kind of busy today, so you don't ask too many questions about the differences between A, B, and C.  Maybe they were shown three different versions of an ad.  Who cares?

(HINT: you will care later)

Brian gives you a CSV of results called `clicks.csv`.  It has the following columns:
- `user_id`: a unique id for each visitor to the FarmBerg site
- `ab_test_group`: either `A`, `B`, or `C` depending on which group the visitor was assigned to
- `click_day`: only filled in *if* the user clicked on a link to purchase

Load `clicks.csv` into the variable `df`.

In [2]:
df = pd.read_csv('clicks.csv') # we are loading the csv into a pandas dataframe, named df
print(df)

                                   user_id group  click_day
0     8e27bf9a-5b6e-41ed-801a-a59979c0ca98     A        NaN
1     eb89e6f0-e682-4f79-99b1-161cc1c096f1     A        NaN
2     7119106a-7a95-417b-8c4c-092c12ee5ef7     A        NaN
3     e53781ff-ff7a-4fcd-af1a-adba02b2b954     A        NaN
4     02d48cf1-1ae6-40b3-9d8b-8208884a0904     A   Saturday
5     5a3ca2d6-25d5-4909-8f07-519f71ee55e8     A        NaN
6     6b929341-1336-4c34-965b-92e368ab160b     A        NaN
7     90b0a07b-e20e-4e0a-872e-5cc303c5676b     A        NaN
8     4b16c922-b2ab-48a8-885c-713ebf0ae159     A        NaN
9     5eb5fc03-fbda-4149-b909-4f5fbc6b152f     A        NaN
10    389ff492-4635-4535-8e42-685f771fccb1     A        NaN
11    1d25885b-56c7-4fdd-bee8-6348c1386bf0     A   Thursday
12    4546807b-8211-4e7b-94cf-4f9c879e284b     A        NaN
13    c447f7f4-a419-41c6-af31-0a17f7943816     A        NaN
14    e75754d9-9342-4f1f-84e1-75c340ef78b2     A        NaN
15    0d38042c-2296-4526-b21c-6b00805688

Define a new column called `is_purchase` which is `Purchase` if `click_day` is not `None` and `No Purchase` if `click_day` is `None`.  This will tell us if each visitor clicked on the Purchase link.

In [3]:
def purchase_value(click):
    if pd.isnull(click):
        return 'No Purchase'
    else:
        return 'Purchase'

df['is_purchase'] = df['click_day'].apply(purchase_value)
print(df)

                                   user_id group  click_day  is_purchase
0     8e27bf9a-5b6e-41ed-801a-a59979c0ca98     A        NaN  No Purchase
1     eb89e6f0-e682-4f79-99b1-161cc1c096f1     A        NaN  No Purchase
2     7119106a-7a95-417b-8c4c-092c12ee5ef7     A        NaN  No Purchase
3     e53781ff-ff7a-4fcd-af1a-adba02b2b954     A        NaN  No Purchase
4     02d48cf1-1ae6-40b3-9d8b-8208884a0904     A   Saturday     Purchase
5     5a3ca2d6-25d5-4909-8f07-519f71ee55e8     A        NaN  No Purchase
6     6b929341-1336-4c34-965b-92e368ab160b     A        NaN  No Purchase
7     90b0a07b-e20e-4e0a-872e-5cc303c5676b     A        NaN  No Purchase
8     4b16c922-b2ab-48a8-885c-713ebf0ae159     A        NaN  No Purchase
9     5eb5fc03-fbda-4149-b909-4f5fbc6b152f     A        NaN  No Purchase
10    389ff492-4635-4535-8e42-685f771fccb1     A        NaN  No Purchase
11    1d25885b-56c7-4fdd-bee8-6348c1386bf0     A   Thursday     Purchase
12    4546807b-8211-4e7b-94cf-4f9c879e284b     A   

We want to count the number of users who made a purchase from each group.  Use `groupby` to count the number of `Purchase` and `No Purchase` from each `group`.  Save your answer to the variable `purchase_counts`.

**Hint**: Group by `group` and `is_purchase` and the function `count` on the column `user_id`.

In [4]:
purchase_counts = df.groupby(by=['group', 'is_purchase'])['user_id'].count()
print(purchase_counts)

group  is_purchase
A      No Purchase    1350
       Purchase        316
B      No Purchase    1483
       Purchase        183
C      No Purchase    1583
       Purchase         83
Name: user_id, dtype: int64


This data is *categorical* and there are *more than 2* conditions, so we'll want to use a chi-squared test to see if there is a significant difference between the three conditions.

Start by filling in the contingency table below with the correct values:
```py
contingency = [[groupA_purchases, groupA_not_purchases],
               [groupB_purchases, groupB_not_purchases],
               [groupC_purchases, groupC_not_purchases]]
```

In [5]:
contingency = [[316, 1350], [183, 1483], [83, 1583]] # filling contingency table with values from above

Now import the function `chi2_contingency` from `scipy.stats` and perform the chi-squared test.

Recall that the *p-value* is the second output of `chi2_contingency`.

In [6]:
from scipy.stats import chi2_contingency

In [7]:
chi_values, p, dof, ex = chi2_contingency(contingency)
print(p)

2.41262135467e-35


Great! It looks like a significantly greater portion of users from Group A made a purchase.

## Part 2: Testing for Exceeding a Goal

Your day is a little less busy than you expected, so you decide to ask Brian about his test.

**You**: Hey Brian! What was that test you were running anyway?

**Brian**: It was awesome! We are trying to get users to purchase a small FarmBurg upgrade package.  It's called a microtransaction.  We're not sure how much to charge for it, so we tested three different price points: \$0.99, \$1.99, and \$4.99.  It looks like significantly more people bought the upgrade package for \$0.99, so I guess that's what we'll charge.

**You**: Oh no! I should have asked you this before we did that chi-squared test.  I don't think that this was the right test at all.  It's true that more people wanted purchase the upgrade at \$0.99; you probably expected that.  What we really want to know is if each price point allows us to make enough money that we can exceed some target goal.  Brian, how much do you think it cost to build this feature?

**Brian**: Hmm.  I guess that we need to generate a minimum of $1000 per week in order to justify this project.

**You**: We have some work to do!

How many visitors came to the site this week?

Hint: Look at the length of `df`.

In [8]:
num_visitors = len(df['user_id'])
print(num_visitors)

4998


Let's assume that this is how many visitors we generally get each week.  Given that, calculate the percent of visitors who would need to purchase the upgrade package at each price point (\$0.99, \$1.99, \$4.99) in order to generate \$1000 per week.

In [9]:
# Calculate the number of people who would need to purchase a $0.99 upgrade in order to generate $1000.
# Then divide by the number of people who visit the site each week.
num_visitors_needed = 1000 / 0.99
portion_99 = num_visitors_needed / num_visitors
print(portion_99)

0.20210104243717691


In [10]:
# Calculate the number of people who would need to purchase a $1.99 upgrade in order to generate $1000.
# Then divide by the number of people who visit the site each week.
num_visitors_needed = 1000 / 1.99
portion_199 = num_visitors_needed / num_visitors
print(portion_199)

0.10054272965467594


In [11]:
# Calculate the number of people who would need to purchase a $4.99 upgrade in order to generate $1000.
# Then divide by the number of people who visit the site each week.
num_visitors_needed = 1000 / 4.99
portion_499 = num_visitors_needed / num_visitors
print(portion_499)

0.040096198800161346


Note that you need a smaller percentage of purchases for higher price points.

Now, for each group, perform a binomial test using `binom_test` from `scipy.stats`.
- `x` will be the number of purchases for that group
- `n` will be the total number of visitors assigned to that group
- `p` will be the target percent of purchases for that price point (calculated above)

Recall that:
- Group `A` is the \$0.99 price point
- Group `B` is the \$1.99 price point
- Group `C` is the \$4.99 price point

In [12]:
# import the binomial test from scipy.stats here
from scipy.stats import binom_test

In [13]:
# Test group A here
x = contingency[0][0] # Row 0, column 0 of contingency matrix stores num purchases in group A
n = contingency[0][0] + contingency[0][1] # add num non-purchases for total vistors in group A
probA = binom_test(x, n, portion_99)
print(probA)

0.21112872994


In [14]:
# Test group B here
x = contingency[1][0]
n = contingency[1][0] + contingency[1][1]
probB = binom_test(x, n, portion_199)
print(probB)

0.206602092466


In [15]:
# Test group C here
x = contingency[2][0]
n = contingency[2][0] + contingency[2][1]
probC = binom_test(x, n, portion_499)
print(probC)

0.0456236724772


If any of the groups passed the binomial test with $p < 0.05$, then we can be confident that enough people will buy the upgrade package at that price point to justify the feature.

Which price point should Brian go with?  Did this surprise you?

Solution:

Brian should go with the pricepoint of 4.99. Group C, which used pricepoint 4.99, was the only group to achive a p-value of less than 0.05, making its pricepoint the clear choice.

I'm not surprised. The Binomial Test attempts to calculate how likely an event is to occur by random chance. In the case of group C, 4.562% represents the probability that this many people would have bought the product by pure chance. Since this value is less than 0.05 (which is often used as a benchmark), it is unlikely to have happened randomnly, indicating higher relative preference of visitors to buy at this pricepoint.

Another reason I'm not surprised is that even though pricepoint C is higher, fewer visitors need to buy at this price to achieve the target proportion. Concretely, the target proportion for group C is 4.009%, and 4.982% bought (more than required). On the other hand, for group A, the target is 20.210% but only 18.968% bought (less than required). For group B, more people bought than required (10.984% vs. 10.054%), but the margin of difference was less than that for group A. Since data already suggested people tended to by at pricepoint C more than needed, this final result is unsurprising.